In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [2]:
data = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv", sep = ',')
data['Attrition'] = data['Attrition'].map({'Yes':0 ,'No':1})
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,0,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,1,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,0,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,1,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,1,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
data_features = data.copy()
data_labels = data_features.pop('Attrition')



In [4]:
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Do a calculation using is
result = 2*input + 1

# the result doesn't have a value
result

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.__operators__.add')>

In [5]:
inputs = {}

for name, column in data_features.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32
    inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'Age': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Age')>,
 'BusinessTravel': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'BusinessTravel')>,
 'DailyRate': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'DailyRate')>,
 'Department': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'Department')>,
 'DistanceFromHome': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'DistanceFromHome')>,
 'Education': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Education')>,
 'EducationField': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'EducationField')>,
 'EmployeeCount': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'EmployeeCount')>,
 'EmployeeNumber': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'EmployeeNumber')>,
 'EnvironmentSatisfaction': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'EnvironmentSatisfaction')>,
 'Gender': <KerasTensor

In [6]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(data[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 26) dtype=float32 (created by layer 'normalization')>

In [7]:
preprocessed_inputs = [all_numeric_inputs]

In [8]:
for name, input in inputs.items():
    if input.dtype == tf.float32:
        continue
    lookup = preprocessing.StringLookup(vocabulary=np.unique(data_features[name]))
    one_hot = preprocessing.CategoryEncoding(max_tokens=lookup.vocab_size())
    x = lookup(input)
    x = one_hot(x)
    preprocessed_inputs.append(x)

In [9]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)
data_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)


In [10]:
data_features_dict = {name: np.array(value) 
                         for name, value in data_features.items()}


In [11]:
features_dict = {name:values[:1] for name, values in data_features_dict.items()}
data_preprocessing(features_dict)

<tf.Tensor: shape=(1, 71), dtype=float32, numpy=
array([[ 0.446,  0.743, -1.011, -0.892,  0.   , -1.701, -0.661,  1.383,
         0.38 , -0.058,  1.153, -0.108,  0.726,  2.125, -1.151, -0.426,
        -1.584,  0.   , -0.932, -0.422, -2.172, -2.494, -0.165, -0.063,
        -0.679,  0.246,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ]],
      dtype=float32)>

In [12]:
def data_model(preprocessing_head, inputs):
    body = tf.keras.Sequential([
        layers.Dense(64),
        layers.Dense(1)
    ])
    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)
    model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam())
    return model

data_model = data_model(data_preprocessing, inputs)

In [13]:
data_model.fit(x=data_features_dict, y=data_labels, epochs=10)

Epoch 1/10
46/46 [==============================] - 2s 3ms/step - loss: 0.6275
Epoch 2/10
46/46 [==============================] - 0s 6ms/step - loss: 0.3787
Epoch 3/10
46/46 [==============================] - 0s 3ms/step - loss: 0.3158
Epoch 4/10
46/46 [==============================] - 0s 3ms/step - loss: 0.3302
Epoch 5/10
46/46 [==============================] - 0s 3ms/step - loss: 0.2865
Epoch 6/10
46/46 [==============================] - 0s 3ms/step - loss: 0.2874
Epoch 7/10
46/46 [==============================] - 0s 4ms/step - loss: 0.3314
Epoch 8/10
46/46 [==============================] - 0s 5ms/step - loss: 0.3010
Epoch 9/10
46/46 [==============================] - 0s 5ms/step - loss: 0.2879
Epoch 10/10
46/46 [==============================] - 0s 3ms/step - loss: 0.2925


In [14]:
ynew = data_model.predict(data_features_dict)

In [15]:
ynew[:15]

array([[-1.039],
       [ 3.745],
       [-0.761],
       [ 1.79 ],
       [ 0.41 ],
       [ 2.192],
       [ 1.388],
       [ 2.03 ],
       [ 2.636],
       [ 2.559],
       [ 2.837],
       [ 1.193],
       [ 1.387],
       [ 2.699],
       [-2.198]], dtype=float32)